## Метод Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? Ранее в REINFORCE мы использовали отдачу $R_t$ (полученную методом Монте-Карло) в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции - критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое семейство различных алгоритмов.



### REINFORCE Loss

Дано:
- $\pi_\theta(a|s)$: Функция стратегии, параметризованная $ \theta $, представляющая вероятность предпринятия действия $a$ в состоянии $ s $.
- $R_t$: Возврат (совокупное вознаграждение) начиная с времени $t$.

Цель - максимизировать ожидаемый возврат:
$$ J(\theta) = \mathbb{E}_{\pi_\theta}[R_t] $$

Градиент $ J(\theta)$ по параметрам стратегии $ \theta $ есть:
$$ \nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[\nabla_\theta \log \pi_\theta(a|s) \cdot R_t] $$

Функция потерь, которую мы стремимся минимизировать, это отрицательное значение $J(\theta)$, так что потери $(L)$ могут быть выражены как:
$$ L(\theta) = -\mathbb{E}_{\pi_\theta}[\log \pi_\theta(a|s) \cdot R_t] $$

### Actor-Critic Loss

Методы Actor-Critic улучшают подход на основе градиента стратегии, уменьшая дисперсию оценки $ R_t $ с помощью функции критика $ V(s) $ или $Q(s, a)$. Функция потерь в методах Actor-Critic объединяет потери градиента стратегии (потери актера) с потерями функции значения (потери критика).

Потери актера (похоже на REINFORCE, но часто использует преимущество $A_t$ вместо $R_t$:
$$ L_{actor}(\theta) = -\mathbb{E}_{\pi_\theta}[\log \pi_\theta(a|s) \cdot A_t] $$
где  $A_t = R_t - V(s)$ это преимущество, указывающее, насколько действие $a$ лучше по сравнению со средним действием в состоянии $s$.

Потери критика:
$$ L_{critic}(\phi) = \mathbb{E}_{\pi_\theta}[(R_t - V_\phi(s))^2] $$
где $V_\phi(s)$ это аппроксимированная функция значения параметрами $\phi$, оценивающая ожидаемый возврат из состояния $s$.

Общие потери для модели Actor-Critic будут комбинацией $L_{actor}$ и $L_{critic}$, часто с дополнительными условиями для регуляризации или бонуса энтропии для поощрения исследования:
$$ L_{total} = L_{actor}(\theta) + \lambda L_{critic}(\phi) + \text{условия энтропии или регуляризации} $$

Эта формулировка позволяет обновлениям как стратегии, так и функции значения двигаться в направлении, улучшающем ожидаемый возврат, при этом критик помогает стабилизировать обучение актера, предоставляя базовую линию для производительности стратегии.



In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Основной цикл

In [ ]:
def print_mean_reward(step, episode_rewards):
    if not episode_rewards:
        return

    t = min(50, len(episode_rewards))
    mean_reward = sum(episode_rewards[-t:]) / t
    print(f"step: {str(step).zfill(6)}, mean reward: {mean_reward:.2f}")
    return mean_reward


def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x).to(device)
    return x


def run(
        env: gym.Env, hidden_size: int, lr: float, gamma: float, max_episodes: int,
        rollout_size: int, replay_buffer_size: int, critic_batch_size: int, critic_updates_per_actor: int
):
    # Инициализируйте агента `agent`, когда сделаете саму реализацию агента ниже по заданию.
    """<codehere>"""
    states_dim = env.observation_space.shape[0]
    actions_dim = env.action_space.n
    agent =
    """</codehere>"""

    step = 0
    episode_rewards = []

    for i_episode in range(1, max_episodes + 1):
        cumulative_reward = 0
        terminated = False
        state, _ = env.reset()

        while not terminated:
            step += 1

            action = agent.act(state)
            next_state, reward, terminated, truncated, _ = env.step(action)

            agent.append_to_replay_buffer(state, action, reward, next_state, terminated)
            state = next_state
            cumulative_reward += reward
            terminated |= truncated

        episode_rewards.append(cumulative_reward)

        # выполняем обновление
        if agent.update(rollout_size, critic_batch_size, critic_updates_per_actor):
            mean_reward = print_mean_reward(step, episode_rewards)
            if mean_reward >= 200:
                print('Принято!')
                return
            episode_rewards = []

In [ ]:
from collections import deque, namedtuple
from operator import attrgetter

class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()


Transition = namedtuple('Transition', ['loss', 'state', 'action', 'reward', 'next_state', 'done'])

class PrioritizedReplayBuffer:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def softmax(self, xs, temp=1000.):
        if not isinstance(xs, np.ndarray):
            xs = np.array(xs)

        # Обрати внимание, насколько большая температура по умолчанию!
        exp_xs = np.exp((xs - xs.max()) / temp)
        return exp_xs / exp_xs.sum()

    def append(self, loss, state, action, reward, next_state, done):
        sample = Transition(loss, state, action, reward, next_state, done)
        self.buffer.append(sample)

    def sample_batch(self, n_samples):
        # Sample randomly `n_samples` samples from replay buffer weighting by priority (sample's loss)
        # and split an array of samples into arrays: states, actions, rewards, next_actions, dones
        # Also, keep samples' indices (into `indices`) to return them too!
        losses = [sample.loss for sample in self.buffer]
        probs = self.softmax(losses)
        indices = np.random.choice(len(self.buffer), n_samples, p=probs)
        states, actions, rewards, next_states, dones = [], [], [], [], []
        for i in indices:
            _, s, a, r, n_s, done = self.buffer[i]
            states.append(s)
            actions.append(a)
            rewards.append(r)
            next_states.append(n_s)
            dones.append(done)

        batch = np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)
        return batch, indices

    def update_batch(self, indices, batch, new_losses):
        """Updates batches with corresponding indices replacing their loss value."""
        states, actions, rewards, next_states, is_done = batch

        for i in range(len(indices)):
            new_sample = Transition(new_losses[i], states[i], actions[i], rewards[i], next_states[i], is_done[i])
            self.buffer[indices[i]] = new_sample

    def sort(self):
        """Sorts replay buffer to move samples with lesser loss to the beginning
        ==> they will be replaced with the new samples earlier."""
        new_rb = deque(maxlen=self.buffer.maxlen)
        new_rb.extend(sorted(self.buffer, key=attrgetter('loss')))
        self.buffer = new_rb

Попробуйте сначала реализовать без памяти прецедентов, а затем дополните вашу реализацию. Текущей реализацией приоритизированной памяти достаточно, чтобы пользоваться ей по аналогии с AgentBatch, стоит лишь добавить метод выборки всех данных по аналогии с `sample_batch`.

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh()
        )

    def forward(self, state):
        state = to_tensor(state)
        return self.net(state)


class ActorCriticModel(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super().__init__()

        # Инициализируйте сеть агента с двумя головами: softmax-актора и линейного критика
        # self.net, self.actor_head, self.critic_head =
        """<codehere>"""

        """</codehere>"""

    def forward(self, state):
        # Вычислите выбранное действие, логарифм вероятности его выбора и соответствующее значение Q-функции
        """<codehere>"""

        """</codehere>"""

        return action, log_prob, q_value

    def evaluate(self, state):
        # Вычислите значения Q-функции для данного состояния
        """<codehere>"""

        """</codehere>"""
        return q_values


class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_size, lr, gamma, replay_buffer_size):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте модель актор-критика и SGD оптимизатор (например, `torch.optim.Adam)`)
        """<codehere>"""

        """</codehere>"""

        self.actor_batch = ActorBatch()
        self.replay_buffer = PrioritizedReplayBuffer(replay_buffer_size)

    def act(self, state):
        # Произведите выбор действия и сохраните необходимые данные в батч для последующего обучения
        # Не забудьте сделать q_value.detach()
        # self.actor_batch.append(..)
        """<codehere>"""

        """</codehere>"""

        return action

    def evaluate(self, state):
        return self.actor_critic.evaluate(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.q_values) < rollout_size:
            return False

        self.update_actor()
        self.update_critic(critic_batch_size, critic_updates_per_actor)
        self.actor_batch.clear()
        return True

    def update_actor(self):
        q_values = to_tensor(self.actor_batch.q_values)
        logprobs = torch.stack(self.actor_batch.logprobs).to(device)

        # Реализуйте шаг обновления актора. Опционально: сделайте нормализацию отдач
        """<codehere>"""
        # Нормализация отдач

        # Вычислите ошибку `loss` и произведите шаг обновления градиентным спуском
        loss =


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        """</codehere>"""

    def update_critic(self, batch_size, critic_updates_per_actor):
        # Реализуйте critic_updates_per_actor шагов обучения критика.
        """<codehere>"""

        # ограничивает сверху количество эпох для буфера небольшого размера
        critic_updates_per_actor = min(
            critic_updates_per_actor,
            5 * len(self.replay_buffer.buffer) // batch_size
        )

        for _ in range(critic_updates_per_actor):
            # generate batch from replay_buffer

            self.optimizer.zero_grad()

            # compute td loss

            loss.backward()
            self.optimizer.step()

            with torch.no_grad():
                # compute updated losses for the training batch and update batch in replay buffer


        """</codehere>"""

        # re-sort replay buffer to prioritize replacing with new samples those samples
        # that have the least loss
        if len(self.replay_buffer.buffer) >= .75 * (self.replay_buffer.buffer.maxlen):
            self.replay_buffer.sort()

    def append_to_replay_buffer(self, s, a, r, next_s, done):
        # Добавьте новый экземпляр данных в память прецедентов.
        """<codehere>"""

        with torch.no_grad():
          # compute td-loss for sample

        # add to replay buffer

        """</codehere>"""

    def compute_td_loss(
        self, states, actions, rewards, next_states, is_done, check_shapes=False, regularizer=.1
    ):
        """ Считатет td ошибку, используя лишь операции фреймворка torch"""

        # переводим входные данные в тензоры
        states = to_tensor(states)                      # shape: [batch_size, state_size]
        actions = to_tensor(actions, int).long()        # shape: [batch_size]
        rewards = to_tensor(rewards)                    # shape: [batch_size]
        next_states = to_tensor(next_states)            # shape: [batch_size, state_size]
        is_done = to_tensor(is_done, bool)              # shape: [batch_size]

        # Реализуйте шаг обновления критика
        """<codehere>"""
        # получаем значения q для всех действий из текущих состояний
        predicted_qvalues = self.evaluate(states)

        # получаем q-values для выбранных действий
        predicted_qvalues_for_actions = predicted_qvalues[range(states.shape[0]), actions]

        with torch.no_grad():
            predicted_next_qvalues = self.evaluate(next_states)
            next_state_values = torch.max(predicted_next_qvalues, axis=-1)[0]

            assert next_state_values.dtype == torch.float32

            # вычисляем target q-values для функции потерь
            #  target_qvalues_for_actions =
            target_qvalues_for_actions = rewards + self.gamma * next_state_values

            # для последнего действия в эпизоде используем
            # упрощенную формулу Q(s,a) = r(s,a),
            # т.к. s' для него не существует
            target_qvalues_for_actions = torch.where(is_done, rewards, target_qvalues_for_actions)

        losses = (predicted_qvalues_for_actions - target_qvalues_for_actions) ** 2

        # MSE loss для минимизации
        loss = torch.mean(losses)
        # добавляем регуляризацию на значения Q
        loss += regularizer * predicted_qvalues_for_actions.mean()
        """</codehere>"""

        return loss, losses

In [ ]:
from gymnasium.wrappers.time_limit import TimeLimit
env_name = "CartPole-v1"

run(
    env = TimeLimit(gym.make(env_name), 1000),
    max_episodes = 50000,  # количество эпизодов обучения
    hidden_size = 64,  # кол-во переменных в скрытых слоях
    rollout_size = 500,  # через столько шагов стратегия будет обновляться
    lr = 0.01, # learning rate
    gamma = 0.995,  # дисконтирующий множитель,
    replay_buffer_size = 5000,
    critic_batch_size = 64,
    critic_updates_per_actor = 32,
)

step: 000546, mean reward: 21.00
step: 001046, mean reward: 9.84
step: 001548, mean reward: 10.68
step: 002056, mean reward: 15.88
step: 002559, mean reward: 17.96
step: 003061, mean reward: 10.04
step: 003595, mean reward: 76.29
step: 004115, mean reward: 47.27
step: 004640, mean reward: 105.00
step: 005225, mean reward: 146.25
step: 005737, mean reward: 20.48
step: 006240, mean reward: 16.77
step: 006809, mean reward: 189.67
step: 007330, mean reward: 104.20
step: 007959, mean reward: 125.80
step: 008574, mean reward: 153.75
step: 009150, mean reward: 192.00
step: 009692, mean reward: 135.50
step: 010325, mean reward: 126.60
step: 010920, mean reward: 119.00
step: 011458, mean reward: 107.60
step: 012039, mean reward: 116.20
step: 012607, mean reward: 142.00
step: 013240, mean reward: 316.50
Принято!




### Soft Actor-Critic (SAC)

Soft Actor-Critic (SAC) — алгоритм обучения с подкреплением, оптимизирующий стохастическую стратегию для максимизации ожидаемого возврата с дополнительным энтропийным бонусом. Этот бонус поощряет исследование, улучшая устойчивость алгоритма и уменьшая его чувствительность к начальным условиям.

Основные компоненты SAC включают:
- **Стратегию (актер)**, параметризованную $\pi_\theta(a|s)$, определяющую вероятностное распределение действий.
- **Две функции Q-значения (критики)**, $Q_{\phi_1}(s, a)$ и $Q_{\phi_2}(s, a)$, минимизирующие ошибку временной разницы (Temporal Difference, TD).
- **Функцию значения (V)**, $V_\psi(s)$, для оценки ожидаемого возврата из состояния $s$, не зависящего от конкретных действий.

#### Функция потерь для SAC:

1. **Потери критика**:
Критики обновляются для минимизации среднеквадратичной ошибки TD:
$$L_{critic}(\phi_i) = \mathbb{E}_{(s, a, r, s') \sim D}\left[\left(Q_{\phi_i}(s, a) - (r + \gamma (V_{\psi'}(s') - \alpha \log \pi_\theta(a|s)))\right)^2\right],$$
где $\gamma$ — фактор дисконтирования, $\alpha$ — параметр температуры, контролирующий важность энтропийного бонуса, и $D$ — опыт из буфера воспроизведения.

2. **Потери актера**:
Актер обновляется через градиентный подъем, максимизируя оценочное Q-значение минус энтропийный бонус:
$$L_{actor}(\theta) = -\mathbb{E}_{s \sim D, a \sim \pi_\theta}\left[\min_{i=1,2} Q_{\phi_i}(s, a) - \alpha \log \pi_\theta(a|s)\right].$$

3. **Потери функции значения**:
Функция значения обновляется для минимизации разницы между её оценками и минимальным Q-значением, скорректированным на энтропийный бонус:
$$L_{value}(\psi) = \mathbb{E}_{s \sim D}\left[\left(V_\psi(s) - \min_{i=1,2} Q_{\phi_i}(s, a_{\theta}) + \alpha \log \pi_\theta(a_{\theta}|s)\right)^2\right],$$
где $a_{\theta}$ — действие, выбранное согласно текущей стратегии.

4. **Автоматическая настройка параметра температуры** $\alpha$:
Для автоматической настройки $\alpha$, используется дополнительная функция потерь, которая обновляет $\alpha$ для поддержания желаемого уровня энтропии.
